## Automatização do processo de coleta de dados e análise (Vindi, Adyen e Agius)

### Instruções:

1. Ponha o relatório da Adyen na pasta designada (BaseAdyen).
2. Ponha o relatório da Agius na pasta designada (BaseAgius).
3. Ponha suas credenciais e nome de usuário da plataforma Vindi na célula abaixo.
4. Na parte superior dessa página, clique em "Kernel", em seguida clique em "Restart & Run All".
5. Aguarde cerca de 10 min (tempo suficiente para a plataforma Vindi gerar um relatório) e visualize o resultado no fim dessa página.

#### OBS.: por favor, não minimizar ou fechar a tela do navegador. Fechará automaticamente com o fim do procedimento.

In [ ]:
#DIGITE SUAS CREDENCIAIS:
login = " " # nome.sobrenome@empresa.com
senha = " " # Senha de login na plataforma Vindi
usuario = " " # Ex.: João Victor Pinheiro (Usuário cadastrado na Vindi)

#INSERIR INTERVALO DE DATA DO RELATÓRIO DA VINDI (o formato tem de ser "DD/MM/AAAA - DD/MM/AAAA")
iData = " " #Ex.: 18/09/2023 - 09/10/2023

In [ ]:
#Verifica se as bibliotecas estão instaladas, se não instala-as.
import importlib

biblioteca = ['selenium', 'webdriver_manager', 'termcolor', 'colorama', 'json', 'IPython.display']

for i in biblioteca:
    try:
        importlib.import_module(i)
        print(f"A biblioteca {i} já está instalada.")
    except ImportError:
        print(f"A biblioteca {i} não está instalada. Instalando agora...")
        !pip install {i}
        print(f"A biblioteca {i} foi instalada com sucesso.")

In [ ]:
#Funções auxiliares:
from IPython.display import Javascript, display
import json

def popupComPrint(t):    
    print_json = json.dumps(t)
    display(Javascript(f"""
        require(
            ["base/js/dialog"], 
            function(dialog) {{
                dialog.modal({{
                    title: 'Automatização Vindi+Adyen+Agius by JV Pinheiro',
                    body: {print_json},
                    buttons: {{
                        'Sair': {{}}
                    }}
                }});
            }}
        );
    """))

In [ ]:
#Limpando pastas:
import os, glob

pasta = 'BaseVindi'
arquivos = os.listdir(pasta)
for arquivo in arquivos:
    caminho_arquivo = os.path.join(pasta, arquivo)
    if os.path.isfile(caminho_arquivo):
        os.remove(caminho_arquivo)
print(f"Todos os arquivos em {pasta} foram apagados.")


diretorio_atual = os.getcwd()
padrao_zip = os.path.join(diretorio_atual, '*.zip')
arquivos_zip = glob.glob(padrao_zip)
try:
    for arquivo in arquivos_zip:
        os.remove(arquivo)
    print("Arquivos .zip foram apagados.")
except Exception as e:
    print("Não existem arquivos .zip nessa pasta.")

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
import time, os

pasta_atual = os.getcwd()
chrome_options = Options()
chrome_options.add_experimental_option('prefs', {
    'download.default_directory': pasta_atual,
    'download.prompt_for_download': False,
    'download.directory_upgrade': True,
    'safebrowsing.enabled': True
})

#Instala driver manager do Chrome
service = Service(ChromeDriverManager().install())

#Abre navegador para bot
nav = webdriver.Chrome(service = service, options = chrome_options)

#Acessa página da Vindi
nav.get('https://app.vindi.com.br/users/sign_in')

#Encontra espaços de login e senha, preenche e clica em "Entrar na plataforma"
nav.find_element('xpath','//*[@id="user_email"]').send_keys(login)
nav.find_element('xpath','//*[@id="user_password"]').send_keys(senha)
nav.find_element('xpath','//*[@id="signin-form_id"]/div[3]/input').click()

#Entra em relatórios
nav.find_element('xpath','//*[@id="main-menu-inner"]/ul/li[4]/a').click()
nav.find_element('xpath','//*[@id="main-menu-inner"]/ul/li[4]/ul/li[1]/a').click()

#Acessa "Cobranças" e "Editar Filtros"
nav.find_element('xpath','//*[@id="content-wrapper"]/div[4]/div/ul[2]/li[1]/a').click()
nav.find_element('xpath','//*[@id="content-wrapper"]/div[3]/div[3]/div[2]/div/div/a/span[1]/i').click()

#Seleções no Filtro
time.sleep(2)
nav.find_element('xpath','//*[@id="paid_at"]').click()
nav.find_element('xpath','//*[@id="paid_at"]').send_keys(iData)
nav.find_element('xpath','/html/body/div[7]/div[1]/ul/li[4]').click()
nav.find_element('xpath','//*[@id="s2id_select2-status"]/ul').click()
nav.find_element('xpath','//*[@id="select2-drop"]/ul/li[2]').click()

#Aplica filtros
nav.find_element('xpath','//*[@id="views-admin-charges-_search_form-0"]').click()

#Inicia exportação
nav.find_element('xpath','//*[@id="charges_list"]/div[1]/div[1]/a').click()
time.sleep(2)
nav.find_element('xpath','//*[@id="export_export_format_csv_plain"]').click()
time.sleep(2)
nav.find_element('xpath','//*[@id="export-submit"]').click()
time.sleep(3)
nav.find_element('xpath','//*[@id="content-wrapper"]/div[4]/div/div[2]/div[2]/a').click()

#Na página de exportações
nav.find_element('xpath','//*[@id="q_user_name_eq"]').send_keys(usuario)
nav.find_element('xpath','//*[@id="export_search"]/div/div[4]/input').click()
while True:
    try:
        nav.refresh()
        nav.maximize_window()
        time.sleep(1)
        nav.find_element('xpath','//*[@id="panel-tab-exported"]/form/table/tbody/tr[1]/td[8]/a[1]').click()
        break
    except (NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException):
        time.sleep(5)

#Fecha o navegador
time.sleep(10)
nav.close()

In [ ]:
#Descompacta arquivo .zip
import zipfile, os

pasta_atual = os.getcwd()
arquivos = os.listdir(pasta_atual)
arquivos_zip = [arquivo for arquivo in arquivos if arquivo.endswith('.zip')]

pasta_destino = 'BaseVindi'
if not os.path.exists(pasta_destino):
    os.makedirs(pasta_destino)

with zipfile.ZipFile(arquivos_zip[0], 'r') as zip_ref:
    zip_ref.extractall(pasta_destino)

In [ ]:
from termcolor import colored
import pandas as pd
import os

#Base B2C (leitura e tratamento)
df = pd.read_csv("BaseB2C.csv", encoding='latin-1', on_bad_lines='skip', sep=";", usecols = ['CD_PESSOA_TITULAR', 'CPF_TITULAR'])
df = df.rename(columns={'CD_PESSOA_TITULAR':'Código_Cliente', 'CPF_TITULAR':'CPF_Cliente'})
df = df.drop_duplicates(subset='CPF_Cliente')

#Base Vindi (leitura e tratamento)
caminho_pasta = 'BaseVindi'
if not os.path.exists(caminho_pasta):
    os.makedirs(caminho_pasta)
arquivo_na_pasta = os.listdir(caminho_pasta)
df2 = pd.read_csv(caminho_pasta + '/' + arquivo_na_pasta[0], usecols = ['Cobrança.DataDePagamento', 'Cliente.NúmeroDoDocumento', 'Cliente.CódigoExterno'])
df2 = df2.rename(columns={'Cobrança.DataDePagamento':'Data_Pagamento', 'Cliente.NúmeroDoDocumento':'CPF_Cliente' , 'Cliente.CódigoExterno':'Código_Cliente'})
df2 = df2[['Data_Pagamento', 'Código_Cliente', 'CPF_Cliente']]
df2['Data_Pagamento'] = pd.to_datetime(df2['Data_Pagamento']).dt.strftime("%d/%m/%Y")
df2 = df2.drop_duplicates(subset='CPF_Cliente')

#Base Adyen (leitura e tratamento)
caminho_pasta = 'BaseAdyen'
if not os.path.exists(caminho_pasta):
    os.makedirs(caminho_pasta)
arquivo_na_pasta = os.listdir(caminho_pasta)
df3 = pd.read_excel(caminho_pasta + '/' + arquivo_na_pasta[0], usecols = ['Creation Date', 'Shopper Reference'])
df3 = df3.rename(columns={'Shopper Reference':'Código_Cliente', 'Creation Date':'Data_Pagamento'})
df3['Data_Pagamento'] = pd.to_datetime(df3['Data_Pagamento']).dt.strftime("%d/%m/%Y")
df3 = df3.drop_duplicates(subset='Código_Cliente')

# Junção da base Adyen com base B2C pra pegar CPF, com código do cliente como referência.
df3_comCPF = pd.merge(df3, df, on='Código_Cliente', how='left')
df3_comCPF = df3_comCPF.drop_duplicates(subset='Código_Cliente')

#Concatenação da base Vindi com Adyen tratadas
final_AdyenVindi = pd.concat([df2, df3_comCPF])

#Base Agius
caminho_pasta = 'BaseAgius'
if not os.path.exists(caminho_pasta):
    os.makedirs(caminho_pasta)
arquivo_na_pasta = os.listdir(caminho_pasta)
df4 = pd.read_excel(caminho_pasta + '/' + arquivo_na_pasta[0], usecols = ['NR_CPF', 'DATA', 'PRAZO'])
df4 = df4.rename(columns={'NR_CPF':'CPF_Cliente'})
df4['DATA'] = df4['DATA'].dt.strftime("%d/%m/%Y")
df4['PRAZO'] = df4['PRAZO'].dt.strftime("%d/%m/%Y")

#Junção da tabela da Agius com a concatenação Vindi + Adyen:
cobrados = pd.merge(final_AdyenVindi, df4, on='CPF_Cliente', how='inner')

#Obtenção dias, data de pagamento - DATA e data de pagamento - PRAZO:
final_AdyenVindi['Data_Pagamento'] = pd.to_datetime(final_AdyenVindi['Data_Pagamento'], format='%d/%m/%Y')
df4['DATA'] = pd.to_datetime(df4['DATA'], format='%d/%m/%Y')
df4['PRAZO'] = pd.to_datetime(df4['PRAZO'], format='%d/%m/%Y')

final_AdyenVindi.reset_index(drop=True, inplace=True)
df4.reset_index(drop=True, inplace=True)

cobrados['dias [pagamento - acordo]'] = (df4['DATA'] - final_AdyenVindi['Data_Pagamento']).dt.days
cobrados['dias [pagamento - prazo]'] = (df4['PRAZO'] - final_AdyenVindi['Data_Pagamento']).dt.days

In [ ]:
#Print dos resultados:
texto = f"FORAM COBRADAS {cobrados.shape[0]} PESSOAS!"
print(colored(texto, 'red', attrs=['bold', 'blink']))
print(colored("\nListagem abaixo: ", 'red', attrs=['bold', 'blink']))
cobrados.head(cobrados.shape[0])

In [ ]:
popupComPrint(texto)